# ORTOOLS  알고리즘 구현

In [1]:
import pandas as pd
pd.read_csv("../data/final_num_171.csv")
distance_matrix=pd.read_csv("../data/final_distance.csv")

In [2]:

matrix = distance_matrix.values
matrix[[0, 171]] = matrix[[171, 0]]

# 열 교체
matrix[:, [0, 171]] = matrix[:, [171, 0]]
lst=matrix
demands=[0,612, 306, 921, 982, 231, 1068, 173, 151, 35, 612, 122, 122, 140, 414, 45, 182, 224, 122, 306, 45, 45, 136, 224, 133, 60, 140, 61, 538, 136, 758, 224, 196, 61, 45, 820, 97, 35, 61, 285, 97, 224, 232, 232, 98, 98, 52, 605, 97, 133, 140, 140, 140, 97, 136, 97, 136, 136, 97, 133, 98, 61, 215, 125, 136, 136, 94, 94, 136, 280, 94, 97, 173, 1465, 35, 285, 384, 217, 332, 258, 984, 112, 73, 73, 151, 332, 1043, 440, 440, 196, 258, 760, 136, 332, 112, 378, 196, 52, 112, 1010, 151, 71, 950, 160, 378, 432, 280, 605, 210, 35, 35, 35, 125, 217, 173, 71, 322, 322, 436, 479, 1065, 387, 173, 322, 322, 71, 564, 423, 432, 215, 215, 92, 52, 160, 52, 92, 92, 160, 160, 280, 397, 35, 35, 35, 35, 35, 35, 35, 35, 35, 1231, 61, 61, 60, 35, 35, 35, 35, 35, 35, 60, 61, 94, 35, 60, 61, 182, 60, 136, 136, 414, 94]

In [3]:
demand_lst=[]
for i in demands:
    i=demand_lst.append(int(i*1.17))
print(demand_lst)

[0, 716, 358, 1077, 1148, 270, 1249, 202, 176, 40, 716, 142, 142, 163, 484, 52, 212, 262, 142, 358, 52, 52, 159, 262, 155, 70, 163, 71, 629, 159, 886, 262, 229, 71, 52, 959, 113, 40, 71, 333, 113, 262, 271, 271, 114, 114, 60, 707, 113, 155, 163, 163, 163, 113, 159, 113, 159, 159, 113, 155, 114, 71, 251, 146, 159, 159, 109, 109, 159, 327, 109, 113, 202, 1714, 40, 333, 449, 253, 388, 301, 1151, 131, 85, 85, 176, 388, 1220, 514, 514, 229, 301, 889, 159, 388, 131, 442, 229, 60, 131, 1181, 176, 83, 1111, 187, 442, 505, 327, 707, 245, 40, 40, 40, 146, 253, 202, 83, 376, 376, 510, 560, 1246, 452, 202, 376, 376, 83, 659, 494, 505, 251, 251, 107, 60, 187, 60, 107, 107, 187, 187, 327, 464, 40, 40, 40, 40, 40, 40, 40, 40, 40, 1440, 71, 71, 70, 40, 40, 40, 40, 40, 40, 70, 71, 109, 40, 70, 71, 212, 70, 159, 159, 484, 109]


In [4]:
"""Capacited Vehicles Routing Problem (CVRP)."""
import numpy as np 
import pandas as pd
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"] = lst
    data["demands"] = demand_lst
    data["vehicle_capacities"] =[5000,3700 , 3000,1500]*2+[5000,3700,3000,1500,5000,3000]
    data["num_vehicles"] = len(data["vehicle_capacities"])
    data["depot"] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    #print(f"Objective: {solution.ObjectiveValue()}")
    total_distance = 0
    total_load = 0
    routelst=[]

    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data["demands"][node_index]
            plan_output += f" {node_index} Load({route_load}) -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        plan_output += f" {manager.IndexToNode(index)} Load({route_load})\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        plan_output += f"Load of the route: {route_load}\n"
    #    print(plan_output)
        routelst.append(plan_output)
        total_distance += route_distance
        total_load += route_load
    print(f"Total distance of all routes: {total_distance}m")
    #print(f"Total load of all routes: {total_load}")
    return solution.ObjectiveValue(),total_distance,total_load,routelst

def main(first,local_search):
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data["demands"][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data["vehicle_capacities"],  # vehicle maximum capacities
        True,  # start cumul to zero
        "Capacity",
    )

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        first
        #routing_enums_pb2.FirstSolutionStrategy.SAVINGS
    )
    search_parameters.local_search_metaheuristic = (
        local_search
        #routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.FromSeconds(30)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        return (print_solution(data, manager, routing, solution))
    else:
        return "no result","no result","no result","no result"
result=[]

# FirstSolutionStrategy와 LocalSearchMetaheuristic의 문자열과 실제 열거형 값을 매핑하는 딕셔너리 생성
first_solution_strategy_mapping = {
    'GLOBAL_CHEAPEST_ARC': routing_enums_pb2.FirstSolutionStrategy.GLOBAL_CHEAPEST_ARC,
    'LOCAL_CHEAPEST_ARC': routing_enums_pb2.FirstSolutionStrategy.LOCAL_CHEAPEST_ARC,
    'PATH_CHEAPEST_ARC': routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC,
    'EVALUATOR_STRATEGY': routing_enums_pb2.FirstSolutionStrategy.EVALUATOR_STRATEGY,
    'ALL_UNPERFORMED': routing_enums_pb2.FirstSolutionStrategy.ALL_UNPERFORMED,
    'BEST_INSERTION': routing_enums_pb2.FirstSolutionStrategy.BEST_INSERTION,
    'PARALLEL_CHEAPEST_INSERTION': routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION,
    'LOCAL_CHEAPEST_INSERTION': routing_enums_pb2.FirstSolutionStrategy.LOCAL_CHEAPEST_INSERTION,
    'SAVINGS': routing_enums_pb2.FirstSolutionStrategy.SAVINGS,
    'SWEEP': routing_enums_pb2.FirstSolutionStrategy.SWEEP,
    'FIRST_UNBOUND_MIN_VALUE': routing_enums_pb2.FirstSolutionStrategy.FIRST_UNBOUND_MIN_VALUE,
    'CHRISTOFIDES': routing_enums_pb2.FirstSolutionStrategy.CHRISTOFIDES,
}

local_search_metaheuristic_mapping = {
    'GREEDY_DESCENT': routing_enums_pb2.LocalSearchMetaheuristic.GREEDY_DESCENT,
    'GUIDED_LOCAL_SEARCH': routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH,
    'SIMULATED_ANNEALING': routing_enums_pb2.LocalSearchMetaheuristic.SIMULATED_ANNEALING,
    'TABU_SEARCH': routing_enums_pb2.LocalSearchMetaheuristic.TABU_SEARCH,
    'GENERIC_TABU_SEARCH': routing_enums_pb2.LocalSearchMetaheuristic.GENERIC_TABU_SEARCH,
}

# 문자열 리스트를 실제 열거형 값으로 변환
first_solution_strategies_ordered = [
    'GLOBAL_CHEAPEST_ARC',
    'LOCAL_CHEAPEST_ARC',
    'PATH_CHEAPEST_ARC',
    'EVALUATOR_STRATEGY',
    'ALL_UNPERFORMED',
    'BEST_INSERTION',
    'PARALLEL_CHEAPEST_INSERTION',
    'LOCAL_CHEAPEST_INSERTION',
    'SAVINGS',
    'SWEEP',
    'FIRST_UNBOUND_MIN_VALUE',
    'CHRISTOFIDES'
]

local_search_metaheuristics_ordered = [
    'GREEDY_DESCENT',
    'GUIDED_LOCAL_SEARCH',
    'SIMULATED_ANNEALING',
    'TABU_SEARCH',
    'GENERIC_TABU_SEARCH'
]
matrix_obj=np.zeros((len(first_solution_strategies_ordered),len(local_search_metaheuristics_ordered)))
matrix_dis=np.zeros((len(first_solution_strategies_ordered),len(local_search_metaheuristics_ordered)))
matrix_load=np.zeros((len(first_solution_strategies_ordered),len(local_search_metaheuristics_ordered)))
# 반복문에서 main 함수 실행
final_route=[]
for strategy in range(len(first_solution_strategies_ordered)):
    for heuristic in range(len(local_search_metaheuristics_ordered)):
        first_solution_strategy = first_solution_strategy_mapping[first_solution_strategies_ordered[strategy]]
        local_search_metaheuristic = local_search_metaheuristic_mapping[local_search_metaheuristics_ordered[heuristic]]
        print(f"{strategy} {heuristic} :", end="")
        obj,dis,load,route=main(first_solution_strategy, local_search_metaheuristic)
        if obj=="no result":
            pass
        else:
            matrix_obj[strategy][heuristic]=obj
            matrix_dis[strategy][heuristic]=dis
            matrix_load[strategy][heuristic]=load
            final_route.append(route)
        

0 0 :0 1 :0 2 :0 3 :0 4 :1 0 :1 1 :1 2 :1 3 :1 4 :2 0 :Total distance of all routes: 173587m
2 1 :Total distance of all routes: 169509m
2 2 :Total distance of all routes: 173587m
2 3 :Total distance of all routes: 171754m
2 4 :Total distance of all routes: 173520m
3 0 :3 1 :3 2 :3 3 :3 4 :4 0 :4 1 :4 2 :4 3 :4 4 :5 0 :5 1 :5 2 :5 3 :5 4 :6 0 :6 1 :6 2 :6 3 :6 4 :7 0 :Total distance of all routes: 181457m
7 1 :Total distance of all routes: 170911m
7 2 :Total distance of all routes: 178366m
7 3 :Total distance of all routes: 178366m
7 4 :Total distance of all routes: 178366m
8 0 :8 1 :8 2 :8 3 :8 4 :9 0 :9 1 :9 2 :9 3 :9 4 :10 0 :10 1 :10 2 :10 3 :10 4 :11 0 :Total distance of all routes: 169614m
11 1 :Total distance of all routes: 164336m
11 2 :Total distance of all routes: 169251m
11 3 :Total distance of all routes: 167470m
11 4 :Total distance of all routes: 169614m


# 알고리즘 구현 결과 

## 1. 목적함수(거리) 결과

In [10]:
df=pd.DataFrame(matrix_dis,index=first_solution_strategies_ordered,columns=local_search_metaheuristics_ordered)
df = df[(df != 0.0).any(axis=1)]
df

,GREEDY_DESCENT,GUIDED_LOCAL_SEARCH,SIMULATED_ANNEALING,TABU_SEARCH,GENERIC_TABU_SEARCH
PATH_CHEAPEST_ARC,173587.0,169509.0,173587.0,171754.0,173520.0
LOCAL_CHEAPEST_INSERTION,181457.0,170911.0,178366.0,178366.0,178366.0
CHRISTOFIDES,169614.0,164336.0,169251.0,167470.0,169614.0


In [11]:
import re
def jingyu(final_route):
    distances = []
    loads = []
    for route in final_route:
        distance_match = re.search(r'Distance of the route: (\d+)m', route)
        load_match = re.search(r'Load of the route: (\d+)', route)
        if distance_match and load_match:
            distances.append(int(distance_match.group(1)))
            loads.append(int(load_match.group(1)))

    # 결과 출력
    distance_A,distance_B,distance_C,distance_D,distance_E=0,0,0,0,0
    # for i, (distance, load) in enumerate(zip(distances, loads)):
    #     print(f"Vehicle {i}: Distance = {distance}m, Load = {load}")
    for i in range(12):
        # if i%5==0: #6100
        #     distance_A+=distances[i]
        if i%4==1:#5000
            distance_B+=distances[i]
        elif i%4==2:#3700
            distance_C+=distances[i]
        elif i%4==3:#3000
            distance_D+=distances[i]
        else:#1500
            distance_E+=distances[i]
    distance_B+=distances[-2]
    distance_D+=distances[-1]
    return np.array([distance_A,distance_B,distance_C,distance_D,distance_E])
# =[5000,3700 , 3000,1500]*2+[5000,3700,3000,1500,5000,3000]

## 2. 알고리즘별 차량별 이동량

In [12]:
lst=[]
for i in final_route:
          lst.append(jingyu(i))
pd.DataFrame(lst,columns=["차량1","차량2","차량3","차량4","차량5"])

,차량1,차량2,차량3,차량4,차량5
0,0,50530,37223,45044,40790
1,0,51582,33646,47973,36308
2,0,50530,37223,45044,40790
3,0,50068,35439,45370,40877
4,0,50530,37223,44977,40790
5,0,53295,37627,48298,42237
6,0,47935,39439,44939,38598
7,0,58397,38334,44443,37192
8,0,58397,38334,44443,37192
9,0,58397,38334,44443,37192


# 경제성분석

In [13]:
oil_1=1490/5600
oil_2=1490/8200
oil_3=1490/6700
oil_4=1490/10600
oil_5=1490/8400
cost_1=(105963636+4287600)/8/312
cost_2=(84916364 +3857660)/8/312
cost_3=(92380000 + 3153620)/8/312
cost_4=(64863636 + 3119110)/8/312
cost_5=(102835455 + 3041100)/8/312
lst=[]
for i in final_route:
          lst.append(jingyu(i))
df2=pd.DataFrame(lst,columns=["차량1","차량2","차량3","차량4","차량5"])
df2.assign(cost=lambda x:x.차량1*oil_1+x.차량2*oil_2+x.차량3*oil_3+x.차량4*oil_4+x.차량5*oil_5+cost_5+cost_2+cost_3+cost_4)

,차량1,차량2,차량3,차량4,차량5,cost
0,0,50530,37223,45044,40790,174523.0
1,0,51582,33646,47973,36308,173535.4
2,0,50530,37223,45044,40790,174523.0
3,0,50068,35439,45370,40877,174103.6
4,0,50530,37223,44977,40790,174513.6
5,0,53295,37627,48298,42237,175829.4
6,0,47935,39439,44939,38598,174140.7
7,0,58397,38334,44443,37192,175476.9
8,0,58397,38334,44443,37192,175476.9
9,0,58397,38334,44443,37192,175476.9


## 3.알고리즘별 표준편차

In [14]:
pd.set_option('display.float_format', '{:.1f}'.format)
m=np.zeros([len(df.index),len(df.columns)])
for i in range(len(df.index)):
          for j in range(len(df.columns)):
                m[i][j]=np.std(jingyu(final_route[i*5+j]))
pd.DataFrame(m,columns=df.columns,index=df.index)

,GREEDY_DESCENT,GUIDED_LOCAL_SEARCH,SIMULATED_ANNEALING,TABU_SEARCH,GENERIC_TABU_SEARCH
PATH_CHEAPEST_ARC,17917.2,18252.5,17917.2,17844.9,17909.5
LOCAL_CHEAPEST_INSERTION,18907.6,17437.9,19366.2,19366.2,19366.2
CHRISTOFIDES,18266.2,17344.9,18190.3,17806.7,18266.2
